# PgSTAC Tutorial
This tutorial is designed to run in the docker compose environment defined in the docker-compose.yml file at the root of the pgstac git repository. For instructions on installing Docker and Docker Compose, you can go to https://docs.docker.com/get-docker/. The instructions provided here use the newer Compose V2 `docker compose` rather than `docker-compose` now that Docker Compose is included as part of latest version of Docker.

To get started with this tutorial you should already have checked out the pgstac repository from github using `git clone https://github.com/bitner/pgstac-tutorial

To start the database, a STAC FastAPI instance, and the Jupyter Notebook for this tutorial: 
```
docker compose up -d tutorial
```

- You can then start up the workshop by going to http://localhost:8891
- You can get to a STAC FastAPI PgSTAC instance at http://localhost:8890 (although this will not work until we have installed PgSTAC onto our Database later)


## Tutorial Setup
### Install magic command to use psql
This installs a "magic" command for the Jupyter Notebook that will allow us to run psql commands to help us explore the database.
The `%psql` line magic is the same as running a command using psql in the terminal.
The `%%psql` cell magic runs the rest of the cell as the stdin to psql. This tutorial uses a system call to the psql utility. This is already installed on the Docker image that comes with this Tutorial. On Debian/Ubuntu clients, this can be installed using `sudo apt install Postgres-client`.

In [ ]:
from IPython.core.magic import register_line_cell_magic
from IPython.display import display, HTML, JSON
import orjson
import shlex
from subprocess import Popen, PIPE

@register_line_cell_magic
def psql(line=None, cell=None):
    args = ["-X"] + (shlex.split(line) or ["-H"])
    if '-1' in args:
        args += ['-v', 'ON_ERROR_STOP=1']
    if cell:
        args += ['-f', '/dev/stdin']
    else:
        cell = ''
    r=Popen(['psql', *args], stdin=PIPE, stdout=PIPE, stderr=PIPE, text=True)
    out, err = r.communicate(input=cell)

    if "-H" in args:
        print(err)
        display(HTML(out))
    elif "-At":
        print(err)
        for line in out.strip().split('\n'):
            try:
                display(orjson.loads(line))
            except:
                display(line)
    else:
        print(err)
        print(out)


## Check the standard Postgres environment variables 
Most tools that work with Postgres use the standard environment variables that are used by all of the tools that come standard as part of Postgres. The pypgstac python utility that comes with pgstac and is installable from pip.

In [ ]:
%env PGDATABASE=pgstac
%env PGUSER=adminrole
%env PGPASSWORD=password
%env PGHOST=pgstac
%env PGPORT=5432


Check that we can login to the database. We will use a call out to the command line psql utility using the `-l` option to list all databases.

In [ ]:
%psql -H -l


## Install PgSTAC to the Database
This docker-compose.yml with this tutorial uses the postgis/postgis:15-3.4 docker image as the base Postgres. PgSTAC requires Postgres>=14 and PostGIS>=3.1. We will now install PgSTAC on the database using the command line `pypgstac migrate` tool. Pypgstac is already installed in the docker image running this notebook. To install otherwise `pip install --upgrade pypgstac[psycopg]`.

In [ ]:
!pypgstac migrate --debug


## Check PgSTAC Install
We can use the psql command line utility to login to our database now and to look around. Let's show what schemas are in the database. SQL Commands beginning with `\` are meta commands in psql. In this case `\dn` is asking to show all schemas (also called namespaces which explains the "n"). We can see that we have a "public" schema which is there by default in all Postgres instances as well as a "pgstac" schema that is owned by the "pgstac_admin" role - this schema as well as the pgstac_admin role were created by the `pypgstac migrate` tool.

In [ ]:
%%psql
\dn


Postgres, much like your shell or python, has a configurable path that it uses to find anything in the database. It is controlled by the "SEARCH_PATH" [setting](https://www.Postgres.org/docs/current/config-setting.html) in Postgres. Postgres will search each schema in the order defined by the "SEARCH_PATH" to find database objects (tables, functions, views, etc). By default, the "SEARCH_PATH" is set to search a schema with the same name as the currently logged in role (which is "pgstac" with the docker environment we are using) followed by the "public" schema.

In [ ]:
%%psql
SHOW SEARCH_PATH;


Since we have installed PgSTAC into the "pgstac" schema, we need to make sure that "pgstac" is available in our envrionment. We can do this temporarily using the "SET" command in Postgres `SET SEARCH_PATH TO pgstac, public;`. Or, we can modify the setting at the DATBASE or ROLE level. For this tutorial, we will set the default setting for "SEARCH_PATH" at the DATABASE level.

In [ ]:
%%psql
ALTER DATABASE pgstac SET SEARCH_PATH TO pgstac,public;
SHOW SEARCH_PATH;


## Configuring Postgres Specific Settings
There are many other settings that can be set at the SYSTEM, DATABASE, ROLE, or SESSION level. If something is set at multiple levels, the most specific level would win, so even though we have set the SEARCH_PATH at the DATABASE level, we could override it in a SESSION by using `SET SEARCH_PATH TO ...`.

Out of the box as well as on most hosted services, the default Postgres configuration is extremely conservative and should be adjusted. There is *NO* one-size-fits-all set of settings even for a given database host instance size. PgSTAC comes with a function that can help to determine what a good starting point may be for some of the most important settings. Fine tuning a database can be an entire career though, so it is important to undestand some of the factors where you may want to adjust these settings. The function takes a single argument which is the memory size of the instance.

In [ ]:
%%psql
SELECT check_pgstac_settings('16GB');


### Important Settings That You Should Always Review

#### [effective_cache_size](http://www.Postgres.org/docs/current/static/runtime-config-resource.html#GUC-SHARED-BUFFERS)
This is the amount of memory that is estimated to be left on the system for the OS and all other processes. This is generally 1/2 to 3/4 of the total system memory.

#### [shared_buffers](http://www.Postgres.org/docs/current/static/runtime-config-resource.html#GUC-SHARED-BUFFERS)
This setting is used to tell the database how much memory is available to dedicate to Postgres for caching data. General rule-of-thumb is to set this to 1/4 of the total system memory.

#### [work_mem](http://www.Postgres.org/docs/current/static/runtime-config-resource.html#GUC-WORK-MEM) and [max_connections](http://www.Postgres.org/docs/current/static/runtime-config-connection.html#GUC-MAX-CONNECTIONS)
This is the memory that is allowed to be used per sort operation per connection for things like sorting and complex queries. This setting will really vary with the use of the database and the number of max connections that are needed in the database. In general `max_connections * work_mem` should be less than the setting for `shared_buffers`. If you have individual queries that you know will be doing larger sorts, the `work_mem` setting can be set at run time: `SET work_mem TO '40MB';`

#### [maintenance_work_mem](http://www.Postgres.org/docs/current/static/runtime-config-resource.html#GUC-MAINTENANCE-WORK-MEM)
This is the amount of memory that is made available for operations such as Vacuuming the database and Creating Indexes. This memory will only be used once at a given time, so it is OK to set this significantly higher than work_mem. 1/4 of the `shared_buffers` is a reasonable place to set this.

#### [seq_page_cost](http://www.Postgres.org/docs/current/static/runtime-config-query.html#GUC-SEQUENTIAL-PAGE-COST) and [random_page_cost](https://www.Postgres.org/docs/current/runtime-config-query.html#GUC-RANDOM-PAGE-COST)
These two variables are interpreted together and it is the ratio of `random_page_cost / seq_page_cost` that really matters. Generally, `seq_page_cost` should be left at the default of 1 and random_page_cost should be changed to reflect the nature of the underlying storage. The default for `random_page_cost` is set to 4 which is appropriate for Spinning Hard Disk Drives. For use with Solid State Drives (which is almost always what a modern hosted platform such as RDS uses), this should be set to 1.1. Having `random_page_cost` set too high can lead to wayyyy slower queries as the Postgres query planner will tend to prefer sequential scans over index scans for many queries.

#### [temp_buffers](https://www.Postgres.org/docs/current/runtime-config-resource.html#GUC_TEMP_BUFFERS)
If using Temporary Tables, increasing this setting can help to avoid spilling to disk. 

In [ ]:
%%psql
ALTER SYSTEM SET EFFECTIVE_CACHE_SIZE TO '12GB';
ALTER SYSTEM SET SHARED_BUFFERS TO '4GB';
ALTER SYSTEM SET WORK_MEM TO '128MB';
ALTER SYSTEM SET MAINTENANCE_WORK_MEM TO '512MB';
ALTER SYSTEM SET MAX_CONNECTIONS TO 20;
ALTER SYSTEM SET RANDOM_PAGE_COST TO 1.1;
ALTER SYSTEM SET TEMP_BUFFERS TO '512MB';
SELECT pg_reload_conf();


Note that these settings could also be set in the postgres.conf settings file on the database server or using the configuration editing tools provided by most Database as a Service providers. 

## PgSTAC Roles
PgSTAC installs three roles with different limitations.
- "pgstac_admin" is the owner of the pgstac schema and all items in the schema. This role has the ability to use or modify anything in pgstac and should be used sparingly.
- "pgstac_ingest" has read/write permissions to create items and collections in PgSTAC, but not to modify any of the PgSTAC utilities. This role should be used when you need access to ingest or modify data in the PgSTAC Catalog.
- "pgstac_read" is the primary role that should be used when accessing PgSTAC when not writing any data. It is not, however, a strictly read-only role as there are still cache and statistics tables which the role will write to behind the scenes.

### Assuming a role
The role that we are using so far is an administrative or root user of the database. While you need to use a role with sufficient priviliges to create a schema, you should never use this role when accessing Postgres for working with PgSTAC. The PgSTAC roles are not set up by default to be able to login to the database, but we can use Role Inheritance to be able to assign another role with all the privileges of one of hte PgSTAC roles.

Best practice would be to create a role that is used for ingest or transactional tasks and one that is used when just reading STAC Items and Collections.

In [ ]:
%%psql -U adminrole
CREATE ROLE stacrw WITH LOGIN PASSWORD 'password' IN ROLE pgstac_ingest;
CREATE ROLE stacr WITH LOGIN PASSWORD 'password' IN ROLE pgstac_read;


## PgSTAC Data Layout
PgSTAC does not directly store STAC Items and Collections as JSON. Rather it pulls some of the information out into properly typed separate columns that can more effectively be used for searching through STAC Items. This data layout is intended to be a back-end implementation and particularly for the "items" table these tables should not be used directly for SELECT/INSERT/DELETE.

### Items Table
If we look a little closer at the "items" table, we can see that it is actually a parent partition. No data is actually stored directly in the items table, but rather in children partitions that are created through the use of triggers on the "collections" table. Right now, you can see that we have a foreign key constraint on the "collections" table (so, you must have a collection added to PgSTAC before adding any "items"). As of now, there are no partitions as we have not added any data yet.

#### Items Table Layout
- id: This is the id from the original JSON Item
- geometry: The geojson from the original JSON item has been extracted and saved as a PostGIS Geometry column.
- collection: The Collection id which is set as a Foreign Key Constraint
- datetime: If the Item JSON has properties.datetime set, this comes from that, otherwise it comes from properties.start_datetime
- end_datetime: If the Item JSON has properties.datetime set, this comes from that, otherwise it comes from properties.end_datetime
- content: This is the remainder of the original JSON Item after removing the geometry, id, and collection as well as well as using a form of compression based on the common item_assets stored with a collection. This is discussed further under "Hydration".
- private: This field is currently not used directly by PgSTAC, but it is to provide a place where additional private metadata about an item that is not part of the public STAC record (ie access constraints, etc) could be stored.

Note that we always have a date range that we can use between datetime and end_datetime (where in the case of a "datetime" in the original JSON represents an instant in time.

In [ ]:
%%psql
\d+ items


### Collections Table
You'll notice that the "collections" table is layed out fairly similary to the Items table. These common columns help enable using the same tooling for search for both Items and Collections. As the "collections" table is generally much much smaller than the "items" table, there are fewer concerns for scalability and this table is not partitioned as is the "items" table.

#### Collections Table Layout
- key: This is an integer primary key that is generated upon creation
- id: This is the id from the original JSON Collection
- geometry: The total bounds from the Collection extent.spatial_extent has been extracted and saved as a PostGIS Geometry column.
- datetime: The start of the Collection extent.temporal_extent
- end_datetime: The end of the Collection extent.temporal_extent
- content: The full original Collection JSON
- base_item: This is used internally for "Hydration" process used to help compress Item records
- private: This field is currently not used directly by PgSTAC, but it is to provide a place where additional private metadata about an item that is not part of the public STAC record (ie access constraints, etc) could be stored.
- partition_trunc: This is used to control how finely partitioned the Items for a Collection are in the "items" table.

In [ ]:
%%psql
\d collections


## Loading Data
A number of things must happen when we load data into PgSTAC
- All data must be transformed into the table layout for Items and Collections
- An Items Collection must be loaded prior to any Items for that Collection
 - When adding a collection, it should be configured for the appropriate partitioning strategy.
- For Items, PgSTAC must create any partitions that are needed prior to actually loading the data in place.
- With partitioned data, Postgres allows for the creation of constraints that allow the Postgres query planner to skip partitions based on query predicates. When loading data, if new data must be checked and the constraints must be modified if necessary.

The management of partitions and constraints can take out fairly aggressive locks on the database when done within the same transaction as loading large amounts of data. This adds a lot more overhead to any transaction loading data into the database than normal Postgres or database loads. When planning how to load data into PgSTAC, it can be very beneficial to try to group inserts into larger chunks of data (but not too large, ~10,000 seems to be a happy medium). PgSTAC should be able to reasonably handle concurrent data loads, but as the locks taken out when modifying partitions/constraints are generally at the partition level, it is good to try to avoid concurrent writes of data into the same partitions.

### Pypgstac Loader
To help mitigate some of the issues with data loading, pypgstac comes with a tool to help bulk load data into PgSTAC. When creating pipelines or doing bulk loading of data into PgSTAC, this should be the preferred method. Data can be loaded via the STAC API when exposed by STAC FastAPI, but when loading via an exposed API, the entire load is done as a single transaction and it also requires a double network hop for all the data (Client -> FastAPI -> PgSTAC).

#### Load Collection

In [ ]:
!pypgstac load collections collections.ndjson.gz --debug


Now if we try to run this again, you'll notice that the insert will fail because the Collection already exists. By default, the loader will not do any conflict management when loading data and so will fail if you try to load a Collection or an Item that already exists. There is an option for pypgstac that allows us to either ignore any duplicates, or to update any existing Collection/Item with the new record.

In [ ]:
!pypgstac load collections collections.ndjson.gz --debug


In [ ]:
!pypgstac load collections collections.ndjson.gz --debug --method ignore


In [ ]:
!pypgstac load collections collections.ndjson.gz --debug --method upsert


#### Load Items

In [ ]:
!pypgstac load items items.ndjson.gz --debug --method ignore


#### Partition Strategy
By default, all Items are partitioned by Collection. 

In [ ]:
%%psql
\d+ items
SELECT * FROM partitions WHERE collection='modis-13Q1-061';


This now shows that there is a partition "_items_1" in the "items" table that contains all data for the 'modis-13Q1-061' data. PgSTAC can also be configured per Collection to further sub-partition data using the datetime by year or month. This is controlled by the "partition_trunc" column on the Collections table. There is a trigger on the Collections table that will make sure that all data is repartitioned on any change of that column. It should be noted, that it is FAR BETTER to modify the partition_trunc value before loading any data. Any time we change that value it requires a full lock on that Collection and a full rewrite of all that data.

Sadly, while pulling together this workshop, I just found a bug that effects is affecting the trigger that updates the partition to the new partition strategy, there will be a fix for the bug, but won't be able to get it out before the workshop :-).

For now, we will remove data from our collection, modify the partition strategy and reload our data into the new strategy.

I will note one of the advantages of having partitions is that data management activities like dropping or truncating a table are much faster using partitions that selecting through all of the data to get a subset.

In [ ]:
%%psql
TRUNCATE _items_1;
UPDATE collections SET partition_trunc='month';

In [ ]:
!pypgstac load items items.ndjson.gz --debug --method=ignore

In [ ]:
%%psql
SELECT * FROM partitions;

We can also add a new Item using pypgstac as a library taking an iterator of Items...

In [ ]:
newitem1={
    "type": "Feature",
    "id":"mynewitem1", 
    "collection":"modis-13Q1-061", 
    "geometry": {
        "type": "Point",
        "coordinates": [125.6, 10.1]
    },
    "properties":{"datetime":"2020-01-01 00:00:00+00"}
}
newitem2={
    "type": "Feature",
    "id":"mynewitem2", 
    "collection":"modis-13Q1-061", 
    "geometry": {
        "type": "Point",
        "coordinates": [125.6, 10.1]
    },
    "properties":{"datetime":"2020-02-01 00:00:00+00"}
}

from pypgstac.db import PgstacDB
from pypgstac.load import Loader
with PgstacDB(debug=True) as db:
    loader = Loader(db)
    loader.load_items([newitem1,newitem2])

In [ ]:
%%psql
SELECT * FROM partitions;

## Search

The Search function is the heart of PgSTAC. The Search Function converts a json stac search input (following the POST specification for a search endpoint) into the SQL needed to search through PgSTAC items. It then converts the internal storage format of the STAC Items back into the Item formatted JSON.

By default the Search function will use a limit of 10 and will sort Items by datetime and id in descending order. Search also makes sure to return information that can be used for keyset pagination which can be much faster than traditional limit/offset pagination.

You can control seeing the debugging information by setting the CLIENT_MIN_MESSAGES setting in Postgres to NOTICE. `SET CLIENT_MIN_MESSAGES TO NOTICE`.

### Context Control
The magic trick that PgSTAC has up it's sleeve can be seen in the debugging output below.

In [ ]:
%%psql
SELECT search();

You can see that the query was broken up into two chunks. From 2023-10-01 to 2023-11-01 and then from 2023-09-01 to 2023-10-01. Since we are sorting by datetime descending, we know that we can search the data in small segments and only continue if there are more rows to fetch. If we switch the sort to datetime ascending, we see that we end up running even more smaller queries. And instead we start scanning in chunks from the bottom.

In [ ]:
%%psql
SELECT search('{"sortby":{"field":"datetime","direction":"asc"}}');

Where this really comes into play is when we may have complicated queries, CPU intensive queries such as spatial lookups, or queries with properties that we haven't indexed. As long as we are sorting by datetime, we can break the problem up into smaller chunks and still be performant as we can stop looking through the data as soon as we have received the "limit" of rows that we have requested.

This means that PgSTAC can be incredibly fast for fetching subsets of data that are sorted by time. Ordinarily if we were to run a SQL query like `SELECT * FROM mytable WHERE st_intersects(mygeomcolumn,<geometry>) ORDER BY mycolumn DESC;`, even if we have an index on both mygeomcolumn and mycolumn, Postgres is going to need to decide whether to use the spatial index to subset the data and then manually sort all the records, or it is going to use the mycolumn index to scan through the data in order and run potentially many expensive spatial operations.

In the STAC specification, this is made even tougher if we chose to use the Context Extension which displays the total number of rows returned by a filter along with the number of rows that are returned after applying the limit. As you can see, the chunked approach that PgSTAC takes makes returning a smaller subset very fast, but for calculating that total count, it may need to scan through a ton of data to get the results.

#### Context Control Settings
There are a couple of settings that can be used to help mitigate the limitations of PgSTAC with providing context results.
- context (on, off) controls whether we should calculate the total count at all
- context_estimated_count (integer) if the estimate produced by the Postgres query planner (the results you get if you were to run EXPLAIN on a SQL query) are smaller than this number, then calculate the actual total count, otherwise just return the estimate
- context_estimated_cost (integer) if the cost as reported by the Postgres query planner is higher than this number, return the estimate
- context_stats_ttl (interval) Cache the results of the context calculation for this amount of time. This allows not only repeated identical searches to reuse the total context, but is especially important as paging through data as the total amount of rows is the same regardless of any changes in sorting or paging.

For very large PgSTAC instances, it is recommended to turn context calculation off or to rely on the ability to use estimated counts. Do note that while for simple queries, the estimates can be quite good, as queries get more complicated, they can occasionally be wildly off.



## Indexing 

Now that we have data in our PgSTAC database, we want to make sure that we can query that data quickly. For that to work, we need to have indexes on the key properties that are used most often in our database. PgSTAC provides a mechanism to make sure that we can build and maintain indexes that will work with the way that PgSTAC creates SQL queries from the CQL2 filters that are passed to the Search function.

Creating Indexes in Postgres is a double edged sword. An appropriate Index that is "hot" in memory can speed up a query by several orders of magnitude. But if there are too many indexes or the indexes are too large, they can have some pretty severe side effects.
-  increasing the amount of time required for inserts/updates to data
-  increasing the amount of time required to plan a query and to choose which indexes to use and how to use them
-  increasing the amount of on disk data storage
-  increasing the amount of contention for staying "hot" in memory

When it comes to indexes, it is difficult to fight the urge to index everything because when you try a new index that fits an exact query once the difference in speed compared to not having the index can be drastic. When you look at the sum performance of your database, however, there may be another story. You should make sure to monitor the actual usage of your STAC instance and to try to match your indexing strategy to the most common use cases for your data.

Given the focus on STAC on SpatioTemporal Data, PgSTAC does automatically include indexes on the id, collection, datetime, end_datetime, and geometry columns. 

### Queryables
In addition to providing a place to store information to be returned by a STAC API queryables endpoint, PgSTAC uses the "queryables" table to define what properties of an Item for each Collection to build additional indexes on. Postgres does have the ability to build indexes on JSON fields, but these indexes are limited to help only in cases of equality comparisons and they can become very unwieldly with Items that have many or deeply nested properties.

The "queryables" table contains the following information:
- id - an internal id
- name - the property name
- collection_ids - an array of collection ids that this entry should apply to (NULL implies that this record applies to all collections)
- definition - the json definition to be returned by the queryables endpoint
- property_wrapper - a wrapper to cast the json property into a Postgres data type. It is possible to create additional custom wrappers.
  - to_text (the default)
  - to_int
  - to_float
  - to_timestamp
- property_index_type - If this is set, then a Postgres Index will be built of this type. For most cases, this will either be NULL to set no index, or BTREE which is the default Postgres Index type.

In [ ]:
%%psql
SELECT id, name, collection_ids, definition, property_wrapper, property_index_type FROM queryables;

Note that in this setup, eo:cloud_cover has been set up to use the to_int wrapper even though the data values can be a float. This allows for smaller indexes and a bit faster ability to search and sort on eo:cloud_cover, but at the expense of a loss of precision in Search.

Let's add a new index that will apply only to the modis Collection that we have loaded.

In [ ]:
%%psql
INSERT INTO queryables (name, collection_ids, definition, property_wrapper, property_index_type)
VALUES (
    'modis:tile-id',
    '{modis-13Q1-061}'::text[],
    '{"type":"string"}',
    'to_text',
    'BTREE'
);
SELECT * FROM pgstac_indexes;
    

### CQL2

The STAC specification uses the "CQL2" (Common Query Lanbguage) as the standard for specifying arbitarily nested queries. CQL2 comes in two flavors CQL2-Text which is a very SQL like way of specifying a query and CQL2-JSON which is much more directly machine usable, but not as fun to write by hand. PgSTAC can only parse the JSON variant of CQL2 and also has legacy support for the older CQL-JSON as well as the "query" parameter from early in development of the STAC specification. For PgSTAC, the CQL-JSON and query variants should be considered deprecated and CQL2-JSON should only be used going forward.

The pygeofilter python library can be used to translate CQL2-Text into CQL2-JSON and is what is used by the stac-fastapi-pgstac library.

In [ ]:
%%psql

SELECT search($$
    {"filter":
        {
            "op": "or", 
            "args": [
                {"op": "gt", "args":[{"property":"modis:tile-id"}, "51035010"]},
                {"op": "eq", "args":[{"property":"platform"}, "terra"]}
            ]
        }
    }
$$);

## Geometry Search
A common desire is to create on the fly mosaics of imagery based on arbitrary STAC requests. PgSTAC has additional functions which work to make these as efficient as possible. These are the functions that are behind TiTiler PgSTAC and allow it to create infinite possibilities in viewing data.

This is all enabled by the GeometrySearch function.

Rather than just using a set limit and paging through results, the GeometrySearch function allows you to use multiple "escape hatches" for when it returns the results of data. Because GeometrySearch is often used behind URL based API's such as Z/X/Y tile services that allow GET requests only, GeometrySearch uses a hash that is stored internally in PgSTAC's cacheing and statistics "searches" table.


In [ ]:
%%psql
SELECT * FROM search_query('{"collections":["modis-13Q1-061"]}');

We can now use the hash provided in order to fetch data using the GeometrySearch Function along with these parameters:
- geom (geometry) -- a PostGIS geometry that acts as the bounds you are trying to fill
- queryhash (text) -- the hash returned from the search_query function
- fields -- an include/exclude list of which properties to include in the results following the STAC Fields Extension
- _scanlimit (int) -- the absolute limit for how many records to scan through that match the search filters before returning results
- _limit (int) -- the limit on how many results to return
- _timelimit (interval) -- the max amount of time to spend looking for results
- exitwhenfull (boolean) -- once the stack of Item geometries from the sorted results would cover the entire geometry, return immediately regardless of the requested limit
- skipcovered (boolean) -- if the coverage of an Item would be obscured by the Items returned before as sorted, do not return that Item in the results.

This gives a number of dials to control tolerance for how long you are willing to wait for results and how many results you would like to deal with for different scenarios.

There are also wrappers that allow for the direct use of GeoJSON (geojsonsearch or an XYZ (xyzsearch) schema.


CREATE OR REPLACE FUNCTION geometrysearch(
    IN geom geometry,
    IN queryhash text,
    IN fields jsonb DEFAULT NULL,
    IN _scanlimit int DEFAULT 10000,
    IN _limit int DEFAULT 100,
    IN _timelimit interval DEFAULT '5 seconds'::interval,
    IN exitwhenfull boolean DEFAULT TRUE, -- Return as soon as the passed in geometry is full covered
    IN skipcovered boolean DEFAULT TRUE -- Skip any items that would show up completely under the previous items
) RETURNS jsonb AS $$

In [ ]:
%%psql
SET CLIENT_MIN_MESSAGES TO WARNING;
SELECT xyzsearch(
    _x=>1,
    _y=>1,
    _z=>1,
    queryhash=>'07c6a95fab72577b195af0691b5e11b9'::text,
    fields=>'{"include":["id"]}'::jsonb,
    exitwhenfull=>TRUE,
    skipcovered=>TRUE
);

## Hydration
PgSTAC uses a method to compress the data stored by an Item by using the "item_assets" attribute on the Item's Collection. When an Item is added, a 'diff' is created between the Assets of an Item and the "item_assets" on the Collection. Any duplicate data in the Item content that can be derived from the Collection is then stripped from the Item content as it is stored in PgSTAC. PgSTAC refers to this process as 'dehydration'.

When a search is returned from PgSTAC, this information is then merged back into the Item pior to be returned. PgSTAC refersw to this process as 'hydration'.

For Collections that have very large Assets, this can save a huge amount of disk space as well as the availability of memory in the Postgres instance. This process is, however, more CPU intensive. There is an option when returning a search that allows for the return of the dehydrated data. This can be done using the '{"conf":{"nohydrate":true}}' parameter as part of the search.

Pypgstac has the ability to do shift this dehydration process away from the Database shifting the load to the application server and speeding things up, particularly under heavy load when multiple application server instances are able to process the results. At PgSTAC version 0.8, this process was sped up even further in pypgstac through implenting the dehydration in Rust. The default setting in STAC FastAPI is to use application side dehydration. 

In [ ]:
%%psql -At
SET CLIENT_MIN_MESSAGES TO WARNING;
select search('{"limit":1}'::jsonb)->'features'->0->'assets'->'hdf';
select content->'assets'->'hdf' from items limit 1;
select content->'item_assets'->'hdf' from collections limit 1;


In [ ]:
%%psql -At
select search()->'features'->0->'assets'->'hdf';


In [ ]:
%%psql -At
select search('{"conf":{"nohydrate":true}}'::jsonb)->'features'->0->'assets'->'hdf';


# Considerations for deployment with STAC-FastAPI
Postgres is limited in communicating over it's own protocol and cannot serve functions data directly over http. PgSTAC has been designed to try to put as much of the heavy lifting of querying a large amount of STAC Items as close to the data itself as possible. It is designed so that building a STAC API should only require a minimal set of wrappers around calls to a PgSTAC Database. STAC-FastAPI-PgSTAC has been built in tandem with PgSTAC and allows to keep the business logic of optimizing queries over large search results close to the data while being able to offload some of the work of serving STAC records onto more easily scalable Application Server instances.

## Fast API
Being built upon Fast API, STAC FastAPI allows the integration of STAC with other related API's (See EOAPI as an example) as well as to add additional capabilities such as authentication, cacheing, or compression without needing to modify STAC-FastAPI itself.

- Deploy using a role that inerhits from pgstac_ingest for the POSTGRES_WRITE_HOST and from pgstac_read for the POSTGRES_READ_HOST
- Make sure to use the USE_API_HYDRATE=true setting
- If deploying using Lambdas or other highly auto-scaling serverless approaches, only use a single connection for the database pool on the application server and deploy alongside a separate connection pooler such as pgbouncer
- If you have default landing queries, make sure that you have some sort of cacheing layer on top of them


